## Baseball Stats

In [ ]:
import os
import sqlite3 as sql
import logging

import pandas as pd
import numpy as np

from scrapers import fangraph
import bb_stat_utils

logging.basicConfig(format='%(asctime)s - %(levelname)s: %(message)s', level=logging.INFO)
from IPython.display import display, HTML
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
# Point to the SQLLTIE Db
db_path = r'/home/zach/dev/demos/2021_Baseball_PURE.db'
if not os.path.exists(db_path):
    #This may take a while (~30 mins)
    fangraphscraper = fangraph.FangraphScraper(db_path)
    fangraphscraper.generate_sqllite_tables()
conn = sql.connect(db_path)

# SQL Query
Since the data is stored on a game by game basis, it must be aggregated per player to be used from a full 2021 season perspective. 
A query in the following style can perform this:
``` 
SELECT 
name, 
id, 
SUM("AB"),
SUM("HR"), 
SUM("RBI")
FROM bat 
WHERE name like 'tatis'
GROUP BY name, id;
``` 

Note the critical inclusion of a `GROUP BY` which groups y player name and ID (the combination of which is unique in this db) and `SUM` which takes the sum of the specified catagory across the grouping. 

See a DB schema for pitching and batting by running the cell below

In [ ]:
pd.read_sql("pragma table_info('BAT')", con=conn)

In [ ]:
pd.read_sql("pragma table_info('PIT')", con=conn)

### Spot Check Batting Data Quality 
According to [this baseball reference link](https://www.baseball-reference.com/players/t/tatisfe02.shtml) Tatis had the following select stats in 2021 Regular Season

| Stat  |Value |
| ------| -----|
| AB    | 478|
| HR    | 42     |
| RBI   | 97 |

Running the code below, our data set agrees


In [ ]:
s = """
SELECT 
name, 
id, 
SUM("AB"),
SUM("HR"), 
SUM("RBI")
FROM bat 
WHERE name like '%tatis%'
GROUP BY name, id;
"""

tatis_df = pd.read_sql(s, con=conn)
tatis_df


### Spot Check Pitching Data Quality 
According to [this baseball reference link](https://www.baseball-reference.com/players/d/darviyu01.shtml) Yu Darvish had the following select stats in 2021 Regular Season

| Stat  |Value |
| ------| -----|
| GS    | 30|
| IP    | 166.1     | 
| H   | 138 |
|ERA | 4.22|

Running the code below, our data set agrees, Note that 0.1 is equivalent to 0.333 for innings pitched, and that ERA is a derived parameter. (ER/(IP/9))


In [ ]:
s = """
SELECT 
name, 
id, 
SUM("GS"),
SUM("IP"), 
SUM("ER"), 
SUM("H")
FROM pit 
WHERE name like '%darvish%'
GROUP BY name, id;
"""
darvish_df = pd.read_sql(s, con=conn)
print(darvish_df)
darvish_df['ERA'] = darvish_df['SUM("ER")'] / ( darvish_df['SUM("IP")'] / 9)
darvish_df


Nice, our bespoke data set agrees with an indepedent data set on baseball reference! Now we can do use to draft the ultimate FBB team!

# TOPSIS - Drafting the ideal FBB Team
We can use the [topsis alogrithm](https://en.wikipedia.org/wiki/TOPSIS) (often used in design probems) to decide who is the ideal candidate for a given Baseball position wheile drafting our team.

TOPSIS relies on comparing numerical, what appears uncompareable. For example TOPSIS allows us to compare a player with 32 HRs and 11 SB to a player with 15 HR and 21 SB. 

As shown in the example above, Roto scoring weights all categories, equally, so our TOPSIS algorithm will use equal weights for all categories. As discussed below this does not imply that stolen bases and home runs are equally as common. 

TOPSIS compares all entries (players in this case) and determines which player have the lowest distance from ideal. 

##### Catagorical Rarity - Modification to TOPSIS
The TOPSIS algortithm presented here has actually been slightly modified from the text book definition to account for scarcity, for example RBIs being more common than HR. To see this look into the `topsis` function itself and note the second normalization step not typically present in textbook definitions

## First Cut - No Knowledge of Team Make Up
The analysis below shows the rankings and relative values of all players for the given position (or in pitching just all pitchers)

The offsensive categories are standard in FBB and are the catagories used in the author's league. (OBP- On Base Percentage and SLUG -Slugging Percentage are derived metrics)



In [22]:
bat_cats = ['HR', 'RBI', 'SB','OBP', 'SLG']
bat_cats_power = {key: 1.0 for key in bat_cats} # This should be negative if a lower value is desired (ie offesnive strikeouts)
unique_pos = ['fDH', 'fC', 'f1B', 'F2B', 'fSS', 'f3B', 'fPH', 'fLF', 'fCF', 'fRF']

bat_raw_cats = ['HR', 'RBI', 'SB', '1B', '2B', '3B', 'HBP', 'BB']
bat_rename_cats =  {f'SUM("{cat}")': cat for cat in bat_raw_cats + unique_pos}


pit_cats = ['ER', 'QS_STAND', 'SV', 'WH', 'SO', 'IP'] # TODO would derive a true QS metric from definition
pit_cats_power = {'ER': -1.0, 'QS_STAND': 1.0, 'SV': 1.0, 'WH': -1.0, 'SO': 1.0, 'IP' : 1.0}

pit_raw_cats = ['ER', 'SV', 'SO', 'IP', 'BB', 'W', 'H', 'GS', 'L']
pit_rename_cats =  {f'SUM("{cat}")': cat for cat in pit_raw_cats}


In [ ]:

# Query independent of position
# Only Keep players with greater than 20 appearances at the given position
query = \
"""SELECT
name,
id,
SUM("{0}"),
SUM("{1}"),
SUM("{2}"),
SUM("{3}"),
SUM("{4}"),
SUM("{5}"),
SUM("{6}"),
SUM("{7}"),
SUM("{8}")
FROM bat
WHERE '{8}' > 1
GROUP BY name, id
HAVING SUM("{8}")>20;
"""

for pos in unique_pos:
    pos_df = pd.read_sql(query.format(*bat_raw_cats,pos), con=conn)
    pos_df.rename(mapper=bat_rename_cats, axis=1, inplace=True)
    bb_stat_utils.calc_SLG_OBP(pos_df)
    if len(pos_df) > 0:
        score_df = bb_stat_utils.topsis(df=pos_df, cats=bat_cats, cats_power=bat_cats_power,  csv_name=None)
        bb_stat_utils.determine_pick_value(score_df)
        display(HTML(f'<h1>{pos}</h1><b>' + score_df.iloc[:10][['name' ,  pos] + bat_cats + ['distance_from_ideals', 'improvement', 'pick_value']].to_html()+'<\b>'))
        
        fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(16,9))
        fig.suptitle(pos)
        ax[0].plot(score_df['distance_from_ideals'].values)
        ax[0].set_ylabel('Distance From Ideal')
        ax[1].hist(score_df['improvement'].values, bins=20)
        ax[1].set_ylabel('Pick Value')
        ax[0].grid()
        ax[1].grid()        

In [26]:
query = \
"""SELECT
name,
id,
SUM("{0}"),
SUM("{1}"),
SUM("{2}"),
SUM("{3}"),
SUM("{4}"),
SUM("{5}"),
SUM("{6}"),
SUM("{7}"),
SUM("{8}")
FROM pit
GROUP BY name, id
HAVING SUM("IP")>20;
"""

pit_df = pd.read_sql(query.format(*pit_raw_cats), con=conn)
pit_df.rename(mapper=pit_rename_cats, axis=1, inplace=True)

bb_stat_utils.calc_WH(pit_df)
score_df = bb_stat_utils.topsis(df=pit_df, cats=pit_cats, cats_power=pit_cats_power, csv_name=None)
bb_stat_utils.determine_pick_value(score_df)

display(HTML(f'<h1>Pitching</h1>' + score_df.iloc[:5][ ['name']+ pit_cats + ['distance_from_ideals']].to_html()))


              name     id  ER  SV   SO          IP  BB   W    H  GS   L
0      aaron-ashby  23550  16   1   39   31.666667  12   3   25   4   2
1     aaron-bummer  16258  22   2   75   56.333333  29   5   42   0   5
2     aaron-civale  19479  53   0   99  124.333333  31  12  108  21   5
3       aaron-loup  10343   6   0   57   56.666667  16   6   37   2   0
4       aaron-nola  16149  93   0  223  180.666667  39   9  165  32   9
..             ...    ...  ..  ..  ...         ...  ..  ..  ...  ..  ..
558       zach-pop  20185  25   0   51   54.666667  24   1   54   0   0
559  zach-thompson  16094  27   0   66   75.000000  28   3   63  14   7
560   zack-greinke   1943  79   0  120  171.000000  36  11  164  29   6
561   zack-littell  15823  20   2   63   61.666667  24   4   46   2   0
562   zack-wheeler  10310  66   0  247  213.333333  46  14  169  32  10

[563 rows x 11 columns]


,name,ER,QS_STAND,SV,WH,SO,IP,distance_from_ideals
291,josh-hader,8,0,34,49,102,58.666667,1.603155
374,max-scherzer,49,26,0,155,236,179.333333,1.613840
224,jacob-degrom,11,13,0,51,146,92.000000,1.622830
126,corbin-burnes,45,23,0,157,234,167.000000,1.627961
537,walker-buehler,57,29,0,201,212,207.666667,1.628640


In [ ]:
pit_raw_cats